In [1]:
from chris_hw3 import load_dataset
train_loader, dev_loader, test_loader, glove_embeddings = load_dataset({
    "-b": 32,
    "-s": 0,
    "emb_dim": 200,
})

Loading ../data/sent140.train.mini.csv
Loading ../data/sent140.dev.csv
Loading ../data/sent140.test.csv
Loading Dev Dataset...
Loading Test Dataset...
Loading Train Dataset...
Allocating loaders...


In [12]:
import torch
import utils
from chris_hw3 import get_model, get_optimizer, get_loss_function, validation_step, N_EPOCHS, DEVICE

def training_step(model, optimizer, loss_function, loader, device):
    epoch_loss = 0
    epoch_acc = 0
    total = 0
    total_batches = (len(loader) + 1)

    # This re-shuffles the dataset, and then sorts by squence length.
    # It gives a bit on randomness on the sequences
    loader.dataset.shuffle_and_sort_by_length()

    #loader.dataset.shuffle()

    model.train()
    for i, batch in enumerate(loader):
        lengths = batch["lengths"]
        x = batch["inputs"]
        y = batch["labels"]
        x, y, lengths = x.to(device), y.to(device), lengths.to(device)

        optimizer.zero_grad()
        y_pred = model(x, lengths)
        loss = loss_function(y_pred, y)

        loss.backward()
        optimizer.step()

        epoch_acc += (torch.argmax(y_pred, dim=1) == y).sum().item()
        epoch_loss += loss.item()
        total += len(y)

        if i % ((total_batches // 10) + 1) == 0:
            print(f"Progress {(i + 1) / total_batches:.2%}: Loss: {epoch_loss / total}, Acc: {epoch_acc / total:.2%}, "
                  f"correct: {epoch_acc}/{total}")

    accuracy = epoch_acc / total
    loss = epoch_loss / (len(loader) + 1)
    return accuracy, loss

cmd_args = utils.read_cmd_args(defaults={
        "-m": 1,
        "-l": 1,
        "-o": 1,
        "-lr": 0.01,
        "-h": 15,
        "-e": 10,
        "-rnn_l": 1,
        "-rrn_t": "lstm"
})

model = get_model(cmd_args, glove_embeddings)
optimizer = get_optimizer(model, cmd_args)
loss_function = get_loss_function(cmd_args)
epochs = cmd_args.get("-e", N_EPOCHS)
print("Command line arguments: ", cmd_args)

history = []
for epoch in range(epochs):
    print(f"\n\n--- Beginning Epoch {epoch} ---")
    train_acc, train_loss = training_step(model, optimizer, loss_function, train_loader, DEVICE)

    print("Validating...")
    valid_acc, valid_loss = validation_step(model, loss_function, dev_loader, DEVICE)
    history.append([epoch, train_acc, train_loss, valid_acc, valid_loss])

    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

print(f"-- END TRAINING --")

# Final Test
print("** TESTING STEP **")
test_accuracy, test_loss = validation_step(model, loss_function, test_loader, DEVICE)
print(f"Final Accuracy: {test_accuracy:.2%}, Loss: {test_loss:.4f}")
utils.save_results(history, test_accuracy, test_loss, cmd_args)

Command line arguments:  {'-m': 1, '-l': 1, '-o': 1, '-lr': 0.01, '-h': 15, '-e': 10, '-rnn_l': 1, '-rrn_t': 'lstm', '-f': '/Users/sauron/Library/Jupyter/runtime/kernel-d990807d-7865-4071-9c3f-b5e1103fe1d5.json'}


--- Beginning Epoch 0 ---
Progress 0.32%: Loss: 0.031076805666089058, Acc: 28.12%, correct: 9/32
Progress 10.51%: Loss: 0.02186860889196396, Acc: 55.87%, correct: 590/1056
Progress 20.70%: Loss: 0.021507509253346, Acc: 56.39%, correct: 1173/2080
Progress 30.89%: Loss: 0.021355166320794636, Acc: 56.64%, correct: 1758/3104
Progress 41.08%: Loss: 0.021201099882754246, Acc: 57.68%, correct: 2381/4128
Progress 51.27%: Loss: 0.02111027495044729, Acc: 58.29%, correct: 3003/5152
Progress 61.46%: Loss: 0.020948100420091435, Acc: 58.99%, correct: 3643/6176
Progress 71.66%: Loss: 0.0208173231780529, Acc: 59.51%, correct: 4285/7200
Progress 81.85%: Loss: 0.020721749036576496, Acc: 59.86%, correct: 4923/8224
Progress 92.04%: Loss: 0.020558916680050762, Acc: 60.46%, correct: 5591/9248
Val